# Segmenting and Clustering Neighborhoods in Toronto

## preparation

In [2]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert address into latitude and longitude

import requests
from pandas.io.json import json_normalize # transform JSON file int pandas dataframe

import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!confa install -c conda-forge folium =0.5.0 --yes
#import folium # map rendering library
print ('Libraries imported')


Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    cer

In [108]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3


---
---
---

# Import DATA

In [9]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup

## get List_of_postal_codes_of_Canada from Wikipedia 

In [10]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'


response = requests.get(url)

soup = BeautifulSoup(response.text, "html.parser")
response.status_code

200

In [112]:
#using BeautifulSoup
soup.select(".wikitable")
print(soup.table.prettify())

<table class="wikitable sortable">
 <tbody>
  <tr>
   <th>
    Postcode
   </th>
   <th>
    Borough
   </th>
   <th>
    Neighbourhood
   </th>
  </tr>
  <tr>
   <td>
    M1A
   </td>
   <td>
    Not assigned
   </td>
   <td>
    Not assigned
   </td>
  </tr>
  <tr>
   <td>
    M2A
   </td>
   <td>
    Not assigned
   </td>
   <td>
    Not assigned
   </td>
  </tr>
  <tr>
   <td>
    M3A
   </td>
   <td>
    <a href="/wiki/North_York" title="North York">
     North York
    </a>
   </td>
   <td>
    <a href="/wiki/Parkwoods" title="Parkwoods">
     Parkwoods
    </a>
   </td>
  </tr>
  <tr>
   <td>
    M4A
   </td>
   <td>
    <a href="/wiki/North_York" title="North York">
     North York
    </a>
   </td>
   <td>
    <a href="/wiki/Victoria_Village" title="Victoria Village">
     Victoria Village
    </a>
   </td>
  </tr>
  <tr>
   <td>
    M5A
   </td>
   <td>
    <a href="/wiki/Downtown_Toronto" title="Downtown Toronto">
     Downtown Toronto
    </a>
   </td>
   <td>
    <a href="

In [12]:
#print(soup.table.prettify())

## geting needed table contet

In [13]:
table= soup.find_all('table', class_= 'wikitable sortable')
table

[<table class="wikitable sortable">
 <tbody><tr>
 <th>Postcode</th>
 <th>Borough</th>
 <th>Neighbourhood
 </th></tr>
 <tr>
 <td>M1A</td>
 <td>Not assigned</td>
 <td>Not assigned
 </td></tr>
 <tr>
 <td>M2A</td>
 <td>Not assigned</td>
 <td>Not assigned
 </td></tr>
 <tr>
 <td>M3A</td>
 <td><a href="/wiki/North_York" title="North York">North York</a></td>
 <td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
 </td></tr>
 <tr>
 <td>M4A</td>
 <td><a href="/wiki/North_York" title="North York">North York</a></td>
 <td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
 </td></tr>
 <tr>
 <td>M5A</td>
 <td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
 <td><a href="/wiki/Regent_Park" title="Regent Park">Harbourfront</a>
 </td></tr>
 <tr>
 <td>M6A</td>
 <td><a href="/wiki/North_York" title="North York">North York</a></td>
 <td><a href="/wiki/Lawrence_Heights" title="Lawrence Heights">Lawrence Heights</a>
 </td></tr>
 <tr>


In [16]:
t_table = table[0] 
type(t_table)

bs4.element.Tag

## filtering Data from table

In [17]:
for row in t_table.find_all('tr'):
    for cell in row.find_all('td'):
        print(cell.text)
    


M1A
Not assigned
Not assigned

M2A
Not assigned
Not assigned

M3A
North York
Parkwoods

M4A
North York
Victoria Village

M5A
Downtown Toronto
Harbourfront

M6A
North York
Lawrence Heights

M6A
North York
Lawrence Manor

M7A
Queen's Park
Not assigned

M8A
Not assigned
Not assigned

M9A
Queen's Park
Queen's Park

M1B
Scarborough
Rouge

M1B
Scarborough
Malvern

M2B
Not assigned
Not assigned

M3B
North York
Don Mills North

M4B
East York
Woodbine Gardens

M4B
East York
Parkview Hill

M5B
Downtown Toronto
Ryerson

M5B
Downtown Toronto
Garden District

M6B
North York
Glencairn

M7B
Not assigned
Not assigned

M8B
Not assigned
Not assigned

M9B
Etobicoke
Cloverdale

M9B
Etobicoke
Islington

M9B
Etobicoke
Martin Grove

M9B
Etobicoke
Princess Gardens

M9B
Etobicoke
West Deane Park

M1C
Scarborough
Highland Creek

M1C
Scarborough
Rouge Hill

M1C
Scarborough
Port Union

M2C
Not assigned
Not assigned

M3C
North York
Flemingdon Park

M3C
North York
Don Mills South

M4C
East York
Woodbine Heights

M5

## make dataframe 

In [90]:
data = pd.read_html(url)
type(data)

list

In [20]:
toro_table = data [0]

In [21]:
toro_table.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


## remove Not assigned from Borough

In [89]:
# Drop not assigned Borough
cleaned= toro_table.loc[toro_table['Borough'] != 'Not assigned']
cleaned.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


## save to csv file

In [23]:
cleaned.to_csv('cleaned_tor', index=False)

In [24]:
x= pd.read_csv('cleaned_tor')
x

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
5,M7A,Queen's Park,Not assigned
6,M9A,Queen's Park,Queen's Park
7,M1B,Scarborough,Rouge
8,M1B,Scarborough,Malvern
9,M3B,North York,Don Mills North


## check duplicates

In [68]:
x[x.duplicated(['Neighbourhood'], keep=False)].sort_values("Neighbourhood")


,Postcode,Borough,Neighbourhood
114,M6N,York,Runnymede
144,M6S,West Toronto,Runnymede
26,M5C,Downtown Toronto,St. James Town
189,M4X,Downtown Toronto,St. James Town


In [88]:
x=x[x.Borough != 'Not assigned']
x = x.sort_values(by=['Postcode','Borough'])

x.reset_index(inplace=True)
x.drop('index',axis=1,inplace=True)

x.shape

(210, 3)

## join similars 

In [85]:

dfjoin=x.groupby(['Postcode','Borough'],as_index=False, sort=False).agg(','.join)

dfjoin.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Not assigned


## after cleanig we got 103 rows

In [86]:
dfjoin.shape

(103, 3)

---

# Merging with geo-coordinates 

## importing Geospatial_Coordinates

In [93]:

body = client_9e7fd777ec7645ebad66df6fcf74ad29.get_object(Bucket='canada-donotdelete-pr-eegamya6wozwqa',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

# If you are reading an Excel file into a pandas DataFrame, replace `read_csv` by `read_excel` in the next statement.
geo_data = pd.read_csv(body)
geo_data.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [94]:
geo_data.shape

(103, 3)

## merging 2 dataframes

In [110]:
#MERGE 2 Dataframes
merged = dfjoin.join(geo_data.set_index('Postal Code'), on='Postcode', how='right')
merged.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Not assigned,43.662301,-79.389494


In [111]:
merged.shape

(103, 5)

---